In [2]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import pdb
import pandas as pd
from PIL import Image
import time
import os
import copy
import shutil
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

PyTorch Version:  1.11.0+cu113
Torchvision Version:  0.12.0+cu113


## Download Data

In [3]:
# Mount to GDrive to get data
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Change this to your own Gdrive path to where the datasets lie in
mount_dir = '/content/drive/My\ Drive/Harvard/2021-22\ Spring\ -\ CS\ 282r/CS\ 282r\ Final\ Project'

In [ ]:
# Full segin 95 to 50 dataset
zipped_dataset = os.path.join(mount_dir, "places_validation_waterbird_95_segin_full.zip")
!unzip $zipped_dataset -d '/content/' 

## Training

Code inspired by PyTorch's baseline finetuning [code](https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html).

In [6]:
# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
model_name = "resnet"

# Number of classes in the dataset
num_classes = 12

# Batch size for training (change depending on how much memory you have)
batch_size = 8

# Number of epochs to train for
num_epochs = 20

# Flag for feature extracting. When False, we finetune the whole model,
#   when True we only update the reshaped layer params
feature_extract = True

In [8]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train','val','test50','test55','test60','test65','test70','test75','test80','test85','test90','test95']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [9]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [10]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    # Downloads pretrained Resnet
    model_ft = models.resnet18(pretrained=use_pretrained)
    set_parameter_requires_grad(model_ft, feature_extract)
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs, num_classes)
    input_size = 224

    # Add 4th channel
    pretrained_weights = model_ft.conv1.weight.clone()
    model_ft.conv1 = nn.Conv2d(4,  # Add extra 2d conv for the 4-channel input
                               64, 
                               kernel_size=7, 
                               stride=2, 
                               padding=3, 
                               bias=False)
    
    model_ft.conv1.weight.data[:, :3] = pretrained_weights
    model_ft.conv1.weight.data[:, 3] = model_ft.conv1.weight.data[:, 0]

    return model_ft, input_size

# Initialize the model for this run
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)

# Print the model we just instantiated
print(model_ft)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

ResNet(
  (conv1): Conv2d(4, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

## Finetuning on Resnet50: 50 to 95 full


In [11]:
data_dir = "places_validation_waterbird_95_segin_full"

In [12]:
# Data augmentation and normalization for training
# Just normalization for validation
test_transform = transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406, 0.5], [0.229, 0.224, 0.225, 0.5])
    ])
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406, 0.5], [0.229, 0.224, 0.225, 0.5]) # alpha channel mean & std?
    ]),
    'val': test_transform,
    'test50': test_transform,
    'test55': test_transform,
    'test60': test_transform,
    'test65': test_transform,
    'test70': test_transform,
    'test75': test_transform,
    'test80': test_transform,
    'test85': test_transform,
    'test90': test_transform,
    'test95': test_transform
}

print("Initializing Datasets and Dataloaders...")

# Create new loader that reads PNG alpha channel
def png_loader(path):
  return Image.open(path).convert('RGBA')

# Create training and validation datasets
image_datasets = {x: datasets.ImageFolder(
                      os.path.join(data_dir, x), 
                      data_transforms[x],
                      loader=png_loader) \
                  for x in [
                            'train', 
                            'val', 
                            'test50',
                            'test55',
                            'test60',
                            'test65',
                            'test70',
                            'test75',
                            'test80',
                            'test85',
                            'test90',
                            'test95'
                            ]}
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(
                        image_datasets[x], 
                        batch_size=batch_size, 
                        shuffle=True, 
                        num_workers=4) \
                    for x in [
                            'train', 
                            'val', 
                            'test50',
                            'test55',
                            'test60',
                            'test65',
                            'test70',
                            'test75',
                            'test80',
                            'test85',
                            'test90',
                            'test95'
                            ]}


# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Initializing Datasets and Dataloaders...


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [13]:
# Test dataloader image channels = 4
dtl = dataloaders_dict['train']
next(iter(dtl))[0].shape

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


torch.Size([8, 4, 224, 224])

In [14]:
# Send the model to GPU
model_ft = model_ft.to(device)

# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

Params to learn:
	 conv1.weight
	 fc.weight
	 fc.bias


In [15]:
# Setup the loss function
criterion = nn.CrossEntropyLoss()

# Train and evaluate
model_ft, hist = train_model(model_ft, 
                             dataloaders_dict, 
                             criterion, 
                             optimizer_ft, 
                             num_epochs=num_epochs, 
                             is_inception=(model_name=="inception"))

Epoch 0/19
----------


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


train Loss: 0.3744 Acc: 0.8430
val Loss: 0.1711 Acc: 0.9324
test50 Loss: 0.2951 Acc: 0.8819
test55 Loss: 0.2710 Acc: 0.8939
test60 Loss: 0.2617 Acc: 0.9037
test65 Loss: 0.2535 Acc: 0.9061
test70 Loss: 0.2458 Acc: 0.9084
test75 Loss: 0.2316 Acc: 0.9146
test80 Loss: 0.2212 Acc: 0.9165
test85 Loss: 0.2124 Acc: 0.9197
test90 Loss: 0.1947 Acc: 0.9310
test95 Loss: 0.1864 Acc: 0.9355

Epoch 1/19
----------
train Loss: 0.3089 Acc: 0.8755
val Loss: 0.1657 Acc: 0.9441
test50 Loss: 0.2804 Acc: 0.8926
test55 Loss: 0.2624 Acc: 0.8952
test60 Loss: 0.2536 Acc: 0.9058
test65 Loss: 0.2450 Acc: 0.9077
test70 Loss: 0.2383 Acc: 0.9096
test75 Loss: 0.2190 Acc: 0.9197
test80 Loss: 0.2104 Acc: 0.9187
test85 Loss: 0.2029 Acc: 0.9277
test90 Loss: 0.1897 Acc: 0.9341
test95 Loss: 0.1783 Acc: 0.9337

Epoch 2/19
----------
train Loss: 0.3276 Acc: 0.8682
val Loss: 0.2149 Acc: 0.9091
test50 Loss: 0.3804 Acc: 0.8367
test55 Loss: 0.3440 Acc: 0.8511
test60 Loss: 0.3310 Acc: 0.8623
test65 Loss: 0.3221 Acc: 0.8695
test70